1. Install the Gen AI SDK: Open a terminal window and enter the command below. You can also [install it in a virtualenv](https://googleapis.dev/python/aiplatform/latest/index.html)

In [ ]:
!pip install --upgrade google-genai
!export GOOGLE_CLOUD_API_KEY="YOUR_API_KEY"

2. Use the following code in your application to request a model response

In [ ]:
from google import genai
from google.genai import types
import base64
import os

def generate():
  client = genai.Client(
      vertexai=True,
      api_key=os.environ.get("GOOGLE_CLOUD_API_KEY"),
  )

  msg1_image1 = types.Part.from_uri(
      file_uri="gs://qwiklabs-gcp-00-1bf1d67fbb2e-bucket/cymbal-product-image.png",
      mime_type="image/png",
  )
  msg2_text1 = types.Part.from_text(text="""**Observing the Scene**

I'm focusing on the image itself, starting with a general overview. The blue backpack and water bottle immediately caught my eye, set against the backdrop of wildflowers, green hills, and a trail. The map suggests a journey.""")
  msg2_text2 = types.Part.from_text(text="""A blue hiking backpack with a turquoise water bottle sits next to a unfolded map on a narrow dirt trail, surrounded by colorful wildflowers and lush green foliage on a sunny hillside.""")
  msg3_image1 = types.Part.from_uri(
      file_uri="gs://qwiklabs-gcp-00-1bf1d67fbb2e-bucket/cymbal-product-image.png",
      mime_type="image/png",
  )
  msg4_text1 = types.Part.from_text(text="""**Imagining a Tagline**

I'm currently focused on crafting a concise and impactful tagline for an outdoor gear advertisement. The image is vivid: a backpack, a map, and a trail winding through wildflowers. I'm exploring phrases that evoke a sense of adventure and the beauty of nature, keeping it short and memorable.


**Generating a Catchphrase**

I'm now zeroing in on the core message. It needs to speak to both functionality and the experience. Thinking about words like \"explore,\" \"discover,\" and \"unleash.\" I'm aiming for a tagline that's not just about the gear, but about the feeling of being in the outdoors.""")
  msg5_image1 = types.Part.from_uri(
      file_uri="gs://qwiklabs-gcp-00-1bf1d67fbb2e-bucket/cymbal-product-image.png",
      mime_type="image/png",
  )

  model = "gemini-2.5-flash"
  contents = [
    types.Content(
      role="user",
      parts=[
        msg1_image1,
        types.Part.from_text(text="""Describe this outdoor product image in one sentence.""")
      ]
    ),
    types.Content(
      role="model",
      parts=[
        msg2_text1,
        msg2_text2
      ]
    ),
    types.Content(
      role="user",
      parts=[
        types.Part.from_text(text="""Generate a catchy tagline suitable for an outdoor gear advertisement based on this image."""),
        msg3_image1
      ]
    ),
    types.Content(
      role="model",
      parts=[
        msg4_text1,
        types.Part.from_text(text="""Explore further, carry lighter.""")
      ]
    ),
    types.Content(
      role="user",
      parts=[
        types.Part.from_text(text="""Write a poetic description inspired by this outdoor scene. Focus on mood and feeling."""),
        msg5_image1
      ]
    ),
  ]
  tools = [
    types.Tool(google_search=types.GoogleSearch()),
  ]

  generate_content_config = types.GenerateContentConfig(
    temperature = 1,
    top_p = 1,
    max_output_tokens = 65535,
    safety_settings = [types.SafetySetting(
      category="HARM_CATEGORY_HATE_SPEECH",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_DANGEROUS_CONTENT",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_HARASSMENT",
      threshold="OFF"
    )],
    tools = tools,
    thinking_config=types.ThinkingConfig(
      thinking_budget=-1,
    ),
  )

  for chunk in client.models.generate_content_stream(
    model = model,
    contents = contents,
    config = generate_content_config,
    ):
    if not chunk.candidates or not chunk.candidates[0].content or not chunk.candidates[0].content.parts:
        continue
    print(chunk.text, end="")

generate()